In [90]:
from web3 import Web3
from solcx import compile_source
import pandas as pd
from datetime import datetime

In [2]:
URL="HTTP://127.0.0.1:7545"

In [69]:
def connect(URL):
    web3=Web3(Web3.HTTPProvider(URL))
    print("Connected?:",web3.isConnected())
    return web3

def get_user_accounts():
    return web3.eth.accounts

def create_default_account(account_hash):
    web3.eth.default_account=account_hash
    
def compile_solidity(file):
    with open(file,'r') as f:
        source=f.read()
    return compile_source(source)

def get_abi_bytecode(compiled_sol):
    contract_id,contract_abi=compiled_sol.popitem()
    abi=contract_abi['abi']
    byte_code=contract_abi['bin']
    return abi,byte_code

def deploy_contract(abi,byte_code):
    contract=web3.eth.contract(abi=abi,bytecode=byte_code)
    tx_hash=contract.constructor().transact()
    tx_receipt=web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt

web3=connect(URL)
print(get_user_accounts())

Connected?: True
['0x4cbc87586da52F809D98F122331C06Ba16D6cC3A', '0xA4a4a85a3aA829384e2de7aD2B16B06EBA07A2bA', '0x58325180563bC75A12543FD380dE90048b89700c', '0xB1ec37fA914Fcf087BC1FB394F53292f337150Be', '0x2ad1fD7BD41DBA854523ecf0E3fbef6E7fC91af1', '0x7107246e4d83cF65F0Fbc56584b274B68745f40d', '0xb40518c06eF3dad45D3F35D3e839cF9b4A230f97', '0x464244D4cfDcE7aB54694dbBcd6C3E1ddCAb8c0a', '0x9108B087b51E0fE9Fa9D37eb794386A98050Bf48', '0x426658F34b041dA97E78ae4972Af6f472b1363e4']


In [70]:
create_default_account('0x4cbc87586da52F809D98F122331C06Ba16D6cC3A')

In [79]:
compiled_sol=compile_solidity('contract.sol')

In [80]:
abi,byte_code=get_abi_bytecode(compiled_sol)

In [81]:
abi

[{'inputs': [],
  'name': 'getUser',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'},
   {'internalType': 'uint256', 'name': '', 'type': 'uint256'},
   {'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256',
    'name': 'invoiceAmount',
    'type': 'uint256'}],
  'name': 'payAmount',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [82]:
byte_code

'608060405234801561001057600080fd5b50610279806100206000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c80635f9362561461003b578063832880e714610057575b600080fd5b61005560048036038101906100509190610172565b610077565b005b61005f61011c565b60405161006e939291906101b9565b60405180910390f35b60004290506000829050600033905060405180606001604052808481526020018381526020018273ffffffffffffffffffffffffffffffffffffffff1681525060008082015181600001556020820151816001015560408201518160020160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555090505050505050565b60008060008060000154600060010154600060020160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16925092509250909192565b60008135905061016c8161022c565b92915050565b60006020828403121561018457600080fd5b60006101928482850161015d565b91505092915050565b6101a4816101f0565b82525050565b6101b381610222565b82525050565b60006060820190506101ce60008301866101aa565b6101db602

In [83]:
receipt=deploy_contract(abi=abi,byte_code=byte_code)

In [84]:
receipt

AttributeDict({'transactionHash': HexBytes('0xe7317711588e1891b3ab7f62b954f22131702650e264430064b0bc899f93ef97'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xabed0d3b83e32438b0dd180b874920031e89674458404f654f9601b8175081af'),
 'blockNumber': 36,
 'from': '0x4cbc87586da52F809D98F122331C06Ba16D6cC3A',
 'to': None,
 'gasUsed': 189883,
 'cumulativeGasUsed': 189883,
 'contractAddress': '0x15c43a3430307dD1DC2e3860Ab71bf344a4008D0',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [85]:
contract=web3.eth.contract(address=receipt['contractAddress'],abi=abi)

In [88]:
contract.functions.payAmount(100).transact()

HexBytes('0xae914f0de4684b6b8a2693979b56cf83d665391e9a3ef09bd52af98727c91737')

In [92]:
data=contract.functions.getUser().call()
data

[1617637684, 100, '0x4cbc87586da52F809D98F122331C06Ba16D6cC3A']

In [96]:
datetime_date=datetime.fromtimestamp(data[0])
pd.to_datetime(datetime_date)

Timestamp('2021-04-05 21:33:04')